# Wokflow  **TEST** con Full Bayesiana 

## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-13 05:40:10 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658525,35.2,1454567,77.7,1121084,59.9
Vcells,1222263,9.4,8388608,64.0,1975165,15.1


In [ ]:
plocal <- list()

# 501
plocal$qcanaritos <- 5L
plocal$min_data_in_leaf <- 2000L
plocal$learning_rate <- 1.0
plocal$gradient_bound <- 0.01


plocal$APO <- 1
plocal$ksemillerio <- 5


In [ ]:
PARAM <- list()
PARAM$experimento <- "comp2_exp12_Z_202108_2"
PARAM$semilla_primigenia <- 907871

In [4]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [5]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset <- fread("~/datasets/competencia_02_crudo.csv.gz" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-11-13 05:40:10 UTC"

Loading required package: data.table



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,767402,41.0,1454567,77.7,1454567,77.7
Vcells,722145115,5509.6,1017395652,7762.2,846000569,6454.5


[1] "2025-11-13 05:40:26 UTC"

In [6]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


### Eliminacion de Features

Completar a gusto LUEGO de realizar un analisis exploratorio de datos.
<br> No necesariamente en esta Segunda Competencia conviele eliminar los mismos campos que en la Primera ...

In [7]:
# Salsa Magica para 202106
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

### Data Quality

In [ ]:
# Data Quality
# Se deben reparar los atributos del dataset que para un cierto mes TODOS sus valores son cero.
# Relevar en forma muy minuciosa en el dataset cuales son los  <atributo,mes> que estan dañados.
# Algunas alternativas de solución son:
# * No hacer absolutamente nada, dejar el valor 0 tal cual está, a sabiendas que es incorrecto
# * Reemplazar esos valores dañados por  NA
# * Interpolar cada valor dañado por el valor del mes previo y el posterior
# * Calcularlo a partir de un modelo, libreria  MICE
AsignarNA <- function(pcampo, pmeses) {

  if( pcampo %in% colnames( dataset ) ) {

    dataset[ foto_mes %in% pmeses, paste0(pcampo) := NA ]
  }
}

AsignarNA("active_quarter", c(202006))
AsignarNA("internet", c(202006))

AsignarNA("mrentabilidad", c(201905, 201910, 202006))
AsignarNA("mrentabilidad_annual", c(201905, 201910, 202006))
AsignarNA("mcomisiones", c(201905, 201910, 202006))
AsignarNA("mactivos_margen", c(201905, 201910, 202006))
AsignarNA("mpasivos_margen", c(201905, 201910, 202006))

AsignarNA("mcuentas_saldo", c(202006))
AsignarNA("ctarjeta_debito_transacciones", c(202006))

AsignarNA("mautoservicio", c(202006))

AsignarNA("ctarjeta_visa_transacciones", c(202006))
AsignarNA("mtarjeta_visa_consumo", c(202006))

AsignarNA("ctarjeta_master_transacciones", c(202006))
AsignarNA("mtarjeta_master_consumo", c(202006))

AsignarNA("ctarjeta_visa_debitos_automaticos", c(201904))
AsignarNA("mttarjeta_visa_debitos_automaticos", c(201904))

AsignarNA("ccajeros_propios_descuentos",
c(201910, 202002, 202006, 202009, 202010, 202102))

AsignarNA("mcajeros_propios_descuentos",
c(201910, 202002, 202006, 202009, 202010, 202102))

AsignarNA("ctarjeta_visa_descuentos",
c(201910, 202002, 202006, 202009, 202010, 202102))

AsignarNA("mtarjeta_visa_descuentos",
c(201910, 202002, 202006, 202009, 202010, 202102))

AsignarNA("ctarjeta_master_descuentos",
c(201910, 202002, 202006, 202009, 202010, 202102))

AsignarNA("mtarjeta_master_descuentos",
c(201910, 202002, 202006, 202009, 202010, 202102))

AsignarNA("ccomisiones_otras", c(201905, 201910, 202006))
AsignarNA("mcomisiones_otras", c(201905, 201910, 202006))

AsignarNA("cextraccion_autoservicio", c(202006))
AsignarNA("mextraccion_autoservicio", c(202006))

AsignarNA("ccheques_depositados", c(202006))
AsignarNA("mcheques_depositados", c(202006))
AsignarNA("ccheques_emitidos", c(202006))
AsignarNA("mcheques_emitidos", c(202006))
AsignarNA("ccheques_depositados_rechazados", c(202006))
AsignarNA("mcheques_depositados_rechazados", c(202006))
AsignarNA("ccheques_emitidos_rechazados", c(202006))
AsignarNA("mcheques_emitidos_rechazados", c(202006))

AsignarNA("tcallcenter", c(202006))
AsignarNA("ccallcenter_transacciones", c(202006))

AsignarNA("thomebanking", c(202006))
AsignarNA("chomebanking_transacciones", c(201910, 202006))

AsignarNA("ccajas_transacciones", c(202006))
AsignarNA("ccajas_consultas", c(202006))

AsignarNA("ccajas_depositos", c(202006, 202105))

AsignarNA("ccajas_extracciones", c(202006))
AsignarNA("ccajas_otras", c(202006))

AsignarNA("catm_trx", c(202006))
AsignarNA("matm", c(202006))
AsignarNA("catm_trx_other", c(202006))
AsignarNA("matm_other", c(202006))

AsignarNA("tmobile_app", c(201901, 201902, 201903, 201904, 201905, 201906, 202006))
AsignarNA("cmobile_app_trx", c(201901, 201902, 201903, 201904, 201905, 201906, 202006))

### Data Drifting

In [ ]:
cat("--- INICIO: Proceso de Ranking con Cero Fijo ---\n")
setorder(dataset, numero_de_cliente, foto_mes)

# --- 1. Definir la función de ranking ---
rank_cero_fijo <- function(x) {
  resultado <- numeric(length(x))
  is_na <- is.na(x)
  resultado[is_na] <- NA_real_
  
  idx_pos <- which(x > 0)
  idx_neg <- which(x < 0)
  
  n_pos <- length(idx_pos)
  n_neg <- length(idx_neg)
  
  if (n_pos > 0) {
    resultado[idx_pos] <- frank(x[idx_pos], ties.method = "random") / n_pos
  }
  if (n_neg > 0) {
    resultado[idx_neg] <- (frank(-x[idx_neg], ties.method = "random") / n_neg) * -1
  }
  
  return(resultado)
}

# --- 2. Automatizar la selección de columnas ---
cat("Identificando columnas a rankear...\n")
patron_regex <- "^(m|Visa_m|Master_m|vm_m)"
cols_a_rankear <- grep(patron_regex, colnames(dataset), value = TRUE)

# Excluir la variable de agrupación
cols_a_rankear <- setdiff(cols_a_rankear, "foto_mes")

cat("Se identificaron", length(cols_a_rankear), 
    "columnas para rankear (basado en el patrón).\n")

# --- 3. Definir los nuevos nombres ---
nuevas_cols_rank <- paste0(cols_a_rankear, "_rank")
cat("Se crearán", length(nuevas_cols_rank), 
    "columnas nuevas (ej: '", nuevas_cols_rank[1], "').\n")

# --- 4. Aplicar la operación ---
cat("Iniciando operación de ranking agrupada por 'foto_mes'...\n")
t_inicio <- Sys.time()

dataset[, (nuevas_cols_rank) := lapply(.SD, rank_cero_fijo), 
        by = foto_mes, 
        .SDcols = cols_a_rankear]

t_fin <- Sys.time()
cat("Operación de ranking completada en", 
    round(difftime(t_fin, t_inicio, units = "secs"), 2), "segundos.\n")

# --- 5. Borrar las columnas originales ---
cat("Eliminando las", length(cols_a_rankear), "columnas originales...\n")
dataset[, (cols_a_rankear) := NULL]

# --- 6. Resumen Final ---
cat("Dimensiones finales del dataset (filas x columnas):", 
    paste(dim(dataset), collapse = " x "), "\n")
cat("Primeras 10 columnas rankeadas:", 
    paste(head(nuevas_cols_rank, 10), collapse = ", "), "\n")

cat("--- FIN: Proceso de Ranking con Cero Fijo ---\n")

Sys.time()

### Feature Engineering Intra-Mes

Crear variables nuevas a partir de las existentes dentro del mismo registro, **sin** ir a buscar información histórica.
<br> El siguiente código es un mínimo ejemplo, agregar nuevos features a gusto

In [ ]:
# Función auxiliar para obtener el nombre correcto (original o rankeado)
get_col_name <- function(base_name, dataset_cols) {
  rank_name <- paste0(base_name, "_rank")
  if (rank_name %in% dataset_cols) {
    return(rank_name)
  } else if (base_name %in% dataset_cols) {
    return(base_name)
  } else {
    return(NULL)  # La columna no existe
  }
}

# Función para verificar si una columna existe (original o rankeada)
col_exists <- function(base_name, dataset_cols) {
  rank_name <- paste0(base_name, "_rank")
  return((base_name %in% dataset_cols) || (rank_name %in% dataset_cols))
}

#------------------------------------------------------------------------------
# FEATURE ENGINEERING INTRA-MES CON SOPORTE PARA RANKS
#------------------------------------------------------------------------------

tryCatch({
  cat("Iniciando Feature Engineering Intra-Mes\n")
  
  # Guardar columnas originales para tracking
  columnas_originales <- copy(colnames(dataset))
  
  # Obtener lista actual de columnas (después del ranking)
  cols_dataset <- colnames(dataset)
  
  #----------------------------------------------------------------------------
  # SECCIÓN 1: FEATURES BÁSICOS (ORIGINALES)
  #----------------------------------------------------------------------------
  cat("Creando features básicos\n")
  
  # El mes 1,2, ..12
  dataset[, kmes := foto_mes %% 100]
  
  # ctr_quarter normalizado por antigüedad
  dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter)]
  dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
  dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
  dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]
  
  # Variable extraída de tesis de maestría - USAR VERSIÓN RANKEADA
  mpayroll_col <- get_col_name("mpayroll", cols_dataset)
  if (!is.null(mpayroll_col)) {
    dataset[, mpayroll_sobre_edad := get(mpayroll_col) / cliente_edad]
  }
  
  #----------------------------------------------------------------------------
  # SECCIÓN 2: RATIOS Y PROPORCIONES
  #----------------------------------------------------------------------------
  cat("Creando ratios y proporciones\n")
  
  # Ratios de actividad con tarjetas
  dataset[, ratio_visa_master := ifelse(ctarjeta_master_transacciones > 0,
                                        ctarjeta_visa_transacciones / ctarjeta_master_transacciones, 0)]
  
  dataset[, ratio_debito_credito := ifelse((ctarjeta_visa_transacciones + ctarjeta_master_transacciones) > 0,
                                           ctarjeta_debito_transacciones / 
                                             (ctarjeta_visa_transacciones + ctarjeta_master_transacciones), 0)]
  
  dataset[, ratio_transacciones_productos := ifelse(cproductos > 0,
                                                    ctrx_quarter / cproductos, 0)]
  
  # Ratio de consumo (USAR VERSIONES RANKEADAS)
  mtarjeta_visa_consumo_col <- get_col_name("mtarjeta_visa_consumo", cols_dataset)
  mtarjeta_master_consumo_col <- get_col_name("mtarjeta_master_consumo", cols_dataset)
  
  if (!is.null(mtarjeta_visa_consumo_col) && !is.null(mtarjeta_master_consumo_col)) {
    dataset[, ratio_consumo_visa_master := ifelse(get(mtarjeta_master_consumo_col) > 0,
                                                  get(mtarjeta_visa_consumo_col) / get(mtarjeta_master_consumo_col), 0)]
  }
  
  # Ratios de rentabilidad (USAR VERSIONES RANKEADAS)
  mrentabilidad_col <- get_col_name("mrentabilidad", cols_dataset)
  mactivos_margen_col <- get_col_name("mactivos_margen", cols_dataset)
  mcomisiones_col <- get_col_name("mcomisiones", cols_dataset)
  
  if (!is.null(mrentabilidad_col)) {
    dataset[, rentabilidad_por_producto := ifelse(cproductos > 0, 
                                                  get(mrentabilidad_col) / cproductos, 0)]
  }
  
  if (!is.null(mrentabilidad_col) && !is.null(mactivos_margen_col)) {
    dataset[, margen_por_activo := ifelse(get(mactivos_margen_col) > 0, 
                                          get(mrentabilidad_col) / get(mactivos_margen_col), 0)]
  }
  
  if (!is.null(mcomisiones_col) && !is.null(mrentabilidad_col)) {
    dataset[, comisiones_sobre_ingresos := ifelse(get(mrentabilidad_col) > 0, 
                                                  get(mcomisiones_col) / get(mrentabilidad_col), 0)]
  }
  
  if (!is.null(mcomisiones_col) && !is.null(mactivos_margen_col)) {
    dataset[, comisiones_sobre_activos := ifelse(get(mactivos_margen_col) > 0,
                                                 get(mcomisiones_col) / get(mactivos_margen_col), 0)]
  }
  
  # Ratios de uso de canales
  dataset[, ratio_digital_tradicional := ifelse(ccajas_transacciones > 0,
                                                chomebanking_transacciones / ccajas_transacciones, 0)]
  
  dataset[, ratio_atm_cajas := ifelse(ccajas_transacciones > 0,
                                      catm_trx / ccajas_transacciones, 0)]
  
  # Ratios de endeudamiento (USAR VERSIONES RANKEADAS)
  visa_msaldototal_col <- get_col_name("Visa_msaldototal", cols_dataset)
  master_msaldototal_col <- get_col_name("Master_msaldototal", cols_dataset)
  visa_mlimitecompra_col <- get_col_name("Visa_mlimitecompra", cols_dataset)
  master_mlimitecompra_col <- get_col_name("Master_mlimitecompra", cols_dataset)
  
  if (!is.null(mpayroll_col) && !is.null(visa_msaldototal_col) && !is.null(master_msaldototal_col)) {
    dataset[, ratio_deuda_ingreso := ifelse(get(mpayroll_col) > 0, 
                                            (get(visa_msaldototal_col) + get(master_msaldototal_col)) / get(mpayroll_col), 0)]
  }
  
  if (!is.null(visa_mlimitecompra_col) && !is.null(visa_msaldototal_col)) {
    dataset[, utilizacion_visa := ifelse(get(visa_mlimitecompra_col) > 0,
                                         get(visa_msaldototal_col) / get(visa_mlimitecompra_col), 0)]
  }
  
  if (!is.null(master_mlimitecompra_col) && !is.null(master_msaldototal_col)) {
    dataset[, utilizacion_master := ifelse(get(master_mlimitecompra_col) > 0,
                                           get(master_msaldototal_col) / get(master_mlimitecompra_col), 0)]
  }
  
  if (col_exists("Visa_msaldototal", cols_dataset) && col_exists("Master_msaldototal", cols_dataset)) {
    dataset[, utilizacion_promedio := (utilizacion_visa + utilizacion_master) / 2]
  }
  
  # Ratios de cuentas (USAR VERSIONES RANKEADAS)
  mcuenta_corriente_col <- get_col_name("mcuenta_corriente", cols_dataset)
  mcaja_ahorro_col <- get_col_name("mcaja_ahorro", cols_dataset)
  
  dataset[, ratio_cc_ca := ifelse(ccaja_ahorro > 0,
                                  ccuenta_corriente / ccaja_ahorro, 0)]
  
  if (!is.null(mcuenta_corriente_col) && !is.null(mcaja_ahorro_col)) {
    dataset[, ratio_saldo_cc_ca := ifelse(get(mcaja_ahorro_col) > 0,
                                          get(mcuenta_corriente_col) / get(mcaja_ahorro_col), 0)]
  }
  
  #----------------------------------------------------------------------------
  # SECCIÓN 3: AGREGACIONES Y TOTALES
  #----------------------------------------------------------------------------
  cat("Creando agregaciones y totales\n")
  
  # Total de productos financieros
  dataset[, total_productos_tarjetas := (ctarjeta_debito + ctarjeta_visa + ctarjeta_master)]
  dataset[, total_productos_cuentas := (ccuenta_corriente + ccaja_ahorro)]
  dataset[, total_productos_prestamos := (cprestamos_prendarios + cprestamos_hipotecarios)]
  dataset[, total_productos_inversiones := (cplazo_fijo + cinversion1 + cinversion2)]
  dataset[, total_productos_seguros := (cseguro_vida + cseguro_auto + cseguro_vivienda + 
                                          cseguro_accidentes_personales)]
  
  # Totales de transacciones por canal
  dataset[, total_trx_digital := (chomebanking_transacciones + catm_trx)]
  dataset[, total_trx_presencial := (ccajas_transacciones + ccallcenter_transacciones)]
  dataset[, total_trx_tarjetas := (ctarjeta_visa_transacciones + ctarjeta_master_transacciones + 
                                     ctarjeta_debito_transacciones)]
  
  # Saldo total en tarjetas (USAR VERSIONES RANKEADAS)
  if (!is.null(visa_msaldototal_col) && !is.null(master_msaldototal_col)) {
    dataset[, saldo_total_tarjetas := (get(visa_msaldototal_col) + get(master_msaldototal_col))]
  }
  
  if (!is.null(visa_mlimitecompra_col) && !is.null(master_mlimitecompra_col)) {
    dataset[, limite_total_tarjetas := (get(visa_mlimitecompra_col) + get(master_mlimitecompra_col))]
  }
  
  visa_mconsumototal_col <- get_col_name("Visa_mconsumototal", cols_dataset)
  master_mconsumototal_col <- get_col_name("Master_mconsumototal", cols_dataset)
  
  if (!is.null(visa_mconsumototal_col) && !is.null(master_mconsumototal_col)) {
    dataset[, consumo_total_tarjetas := (get(visa_mconsumototal_col) + get(master_mconsumototal_col))]
  }
  
  # Totales monetarios de cuentas (USAR VERSIONES RANKEADAS)
  if (!is.null(mcuenta_corriente_col) && !is.null(mcaja_ahorro_col)) {
    dataset[, saldo_total_cuentas := (get(mcuenta_corriente_col) + get(mcaja_ahorro_col))]
  }
  
  mcaja_ahorro_dolares_col <- get_col_name("mcaja_ahorro_dolares", cols_dataset)
  mplazo_fijo_dolares_col <- get_col_name("mplazo_fijo_dolares", cols_dataset)
  
  if (!is.null(mcaja_ahorro_dolares_col) && !is.null(mplazo_fijo_dolares_col)) {
    dataset[, saldo_total_dolares := (get(mcaja_ahorro_dolares_col) + get(mplazo_fijo_dolares_col))]
  }
  
  # Totales de préstamos (USAR VERSIONES RANKEADAS)
  mprestamos_prendarios_col <- get_col_name("mprestamos_prendarios", cols_dataset)
  mprestamos_hipotecarios_col <- get_col_name("mprestamos_hipotecarios", cols_dataset)
  
  if (!is.null(mprestamos_prendarios_col) && !is.null(mprestamos_hipotecarios_col)) {
    dataset[, monto_total_prestamos := (get(mprestamos_prendarios_col) + get(mprestamos_hipotecarios_col))]
  }
  
  # Totales de inversiones (USAR VERSIONES RANKEADAS)
  mplazo_fijo_pesos_col <- get_col_name("mplazo_fijo_pesos", cols_dataset)
  minversion1_pesos_col <- get_col_name("minversion1_pesos", cols_dataset)
  minversion1_dolares_col <- get_col_name("minversion1_dolares", cols_dataset)
  minversion2_col <- get_col_name("minversion2", cols_dataset)
  
  if (!is.null(mplazo_fijo_pesos_col) && !is.null(mplazo_fijo_dolares_col) && 
      !is.null(minversion1_pesos_col) && !is.null(minversion1_dolares_col) && !is.null(minversion2_col)) {
    dataset[, monto_total_inversiones := (get(mplazo_fijo_pesos_col) + get(mplazo_fijo_dolares_col) + 
                                            get(minversion1_pesos_col) + get(minversion1_dolares_col) + get(minversion2_col))]
  }
  
  # Payroll total (USAR VERSIONES RANKEADAS)
  mpayroll2_col <- get_col_name("mpayroll2", cols_dataset)
  
  if (!is.null(mpayroll_col) && !is.null(mpayroll2_col)) {
    dataset[, mpayroll_total := (get(mpayroll_col) + get(mpayroll2_col))]
  }
  
  #----------------------------------------------------------------------------
  # SECCIÓN 4: INDICADORES DE COMPORTAMIENTO
  #----------------------------------------------------------------------------
  cat("Creando indicadores de comportamiento\n")
  
  # Intensidad de uso de productos
  dataset[, intensidad_tarjetas := ifelse(total_productos_tarjetas > 0,
                                          total_trx_tarjetas / total_productos_tarjetas, 0)]
  
  dataset[, intensidad_cuentas := ifelse(total_productos_cuentas > 0,
                                         (chomebanking_transacciones + ccajas_transacciones) / 
                                           total_productos_cuentas, 0)]
  
  # Diversificación
  dataset[, diversificacion_productos := total_productos_tarjetas + total_productos_cuentas + 
            total_productos_prestamos + total_productos_inversiones]
  
  dataset[, diversificacion_canales := (as.numeric(chomebanking_transacciones > 0) +
                                          as.numeric(ccajas_transacciones > 0) +
                                          as.numeric(catm_trx > 0) +
                                          as.numeric(ccallcenter_transacciones > 0))]
  
  # Concentración de actividad
  dataset[, concentracion_visa := ifelse(total_trx_tarjetas > 0,
                                         ctarjeta_visa_transacciones / total_trx_tarjetas, 0)]
  
  dataset[, concentracion_digital := ifelse((total_trx_digital + total_trx_presencial) > 0,
                                            total_trx_digital / (total_trx_digital + total_trx_presencial), 0)]
  
  # Activación de productos vs tenencia
  dataset[, ratio_productos_activos := ifelse(cproductos > 0,
                                              diversificacion_productos / cproductos, 0)]
  
  #----------------------------------------------------------------------------
  # SECCIÓN 5: FEATURES DE RIESGO Y ALERTAS
  #----------------------------------------------------------------------------
  cat("Creando features de riesgo\n")
  
  # Indicadores de problemas financieros (USAR VERSIONES RANKEADAS SI EXISTEN)
  visa_msaldopesos_col <- get_col_name("Visa_msaldopesos", cols_dataset)
  master_msaldopesos_col <- get_col_name("Master_msaldopesos", cols_dataset)
  
  if (!is.null(visa_msaldopesos_col) && !is.null(master_msaldopesos_col)) {
    dataset[, flag_sobregiro := as.numeric(get(visa_msaldopesos_col) < 0 | get(master_msaldopesos_col) < 0)]
  }
  
  if (col_exists("Visa_msaldototal", cols_dataset) && col_exists("Master_msaldototal", cols_dataset)) {
    dataset[, flag_limite_alto := as.numeric(utilizacion_visa > 0.8 | utilizacion_master > 0.8)]
  }
  
  dataset[, flag_cheques_rechazados := as.numeric(ccheques_emitidos_rechazados > 0)]
  dataset[, flag_mora_visa := as.numeric(!is.na(Visa_Finiciomora))]
  dataset[, flag_mora_master := as.numeric(!is.na(Master_Finiciomora))]
  
  # Indicadores de baja actividad
  dataset[, flag_baja_actividad := as.numeric(ctrx_quarter < 5)]
  
  if (col_exists("mpayroll", cols_dataset)) {
    dataset[, flag_sin_payroll := as.numeric(mpayroll_total == 0 | is.na(mpayroll_total))]
  }
  
  dataset[, flag_sin_tarjeta := as.numeric(ctarjeta_visa + ctarjeta_master == 0)]
  
  # Score de riesgo simple
  dataset[, score_riesgo := flag_sobregiro * 3 + 
            flag_limite_alto * 2 + 
            flag_cheques_rechazados * 2 +
            flag_mora_visa * 3 +
            flag_mora_master * 3 +
            flag_baja_actividad * 1]
  
  # Nivel de endeudamiento
  if (col_exists("mpayroll", cols_dataset) && col_exists("Visa_msaldototal", cols_dataset)) {
    dataset[, nivel_endeudamiento := ifelse(ratio_deuda_ingreso < 0.3, 1L,
                                            ifelse(ratio_deuda_ingreso < 0.5, 2L,
                                                   ifelse(ratio_deuda_ingreso < 0.7, 3L,
                                                          ifelse(ratio_deuda_ingreso < 1.0, 4L, 5L))))]
  }
  
  #----------------------------------------------------------------------------
  # RESUMEN
  #----------------------------------------------------------------------------
  nuevas_features <- setdiff(colnames(dataset), columnas_originales)
  cat("Total de nuevas features creadas:", length(nuevas_features), "\n")
  
  # Liberar memoria
  rm(columnas_originales, nuevas_features)
  gc(verbose = FALSE)
  
  cat("Feature Engineering Intra-Mes completado exitosamente\n")
  
}, error = function(e) {
  cat("######################################################\n")
  cat("ERROR en Feature Engineering Intra-Mes\n")
  cat("Mensaje:", e$message, "\n")
  cat("######################################################\n")
  stop(e)
})

Sys.time()

[1] "2025-11-13 05:40:27 UTC"

### Feature Engineering Historico

In [9]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

Loading required package: Rcpp



In [10]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [11]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

In [ ]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 3
dataset[,
  paste0(cols_lagueables, "_lag3") := shift(.SD, 3, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 6
dataset[,
  paste0(cols_lagueables, "_lag6") := shift(.SD, 6, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 12
dataset[,
  paste0(cols_lagueables, "_lag12") := shift(.SD, 12, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta3") := get(vcol) - get(paste0(vcol, "_lag3"))]
  dataset[, paste0(vcol, "_delta6") := get(vcol) - get(paste0(vcol, "_lag6"))]
  dataset[, paste0(vcol, "_delta12") := get(vcol) - get(paste0(vcol, "_lag12"))]
}

Sys.time()

[1] "2025-11-13 05:41:31 UTC"

In [13]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

In [14]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

[1] 921

[1] "2025-11-13 05:42:26 UTC"

## Modelado

No hay modelado, no se hace optimizacion de hiperparametros.

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los treinta meses del intervalo [201901, 202104]
* Se realiza undersampling al 5%
* Se utilizan los hiperparámetros optimos encontrados en la Bayesian Optimization
   * Se escala min_data_in_leaf

### Final Training Strategy

In [ ]:
PARAM$train_final$future <- c(202108)  # Mes sin clase conocida

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106
)

PARAM$train_final$undersampling <- 0.05

# DEFINIR CORTES DE ENVÍOS (BASADO EN ANÁLISIS DE 202106)
PARAM$train_final$envios_produccion <- c(9500, 10000, 10500, 11000, 11500, 12000)

In [16]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

In [17]:
# canaritos
PARAM$train_final$lgbm$qcanaritos <- plocal$qcanaritos

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
  setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )
}

Sys.time()

[1] "2025-11-13 05:42:29 UTC"

#### Registros cambio las proporciones de POS/NEG

In [18]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [19]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Models

Aqui SIEMPRE voy a hacer un semillerio, independientemente de si en la Bayesian Optimization calculé un semillerio en cada iteración.
<br> Entreno un LightGBM para cada semilla,  y guardo el modelo dentro de la carpeta  **modelitos**
<br> Intencionalmente en una primera etapá se generan los modelos y graban, y en una segunda etapa se leen eso modelos y se aplican a los datos del futuro

APO controla cuantas veces se repite el modelo, que se usa para promediar ganancias y reportar en la Pseudo Competencia algo razonable
<br> El modelo puede ser un LightGBM simple (ksemillerio==1)  o un Ensemble Semillerio( ksemillerio > 1 )
<br> Lamentablmente APO necesita utilizar muchas semillas, y eso demanda TIEMPO de corrida

In [20]:

PARAM$train_final$lgbm$param_completo <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= plocal$min_data_in_leaf,  #este ya es el valor default de LightGBM

  num_iterations= 9999L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 9999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= plocal$learning_rate,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale
    
  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...
    
  canaritos= PARAM$train_final$lgbm$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= plocal$gradient_bound   # default de zLightGBM
)

Sys.time()

[1] "2025-11-13 05:42:29 UTC"

In [ ]:
# Semillerio Final
PARAM$train_final$APO <- plocal$APO
PARAM$train_final$ksemillerio  <- plocal$ksemillerio

In [22]:
# detach("package:lightgbm", unload= TRUE)

In [23]:
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")

Loading required package: zlightgbm



In [24]:
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes



In [25]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$APO*PARAM$train_final$ksemillerio )]
PARAM$train_final$semillas

[1] 974411

In [26]:
campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c( "clase_ternaria", "clase01", "training", "azar")
)

In [27]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 235841 columnas 925 


[1] "2025-11-13 05:42:32 UTC"

In [28]:
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$lgbm$param_completo)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}

Sys.time()

[1] "2025-11-13 05:42:32 UTC"

### Scoring

Se hace el predict() del modelo en los datos del futuro

In [ ]:
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]

cat("Registros en 202108:", nrow(dfuture), "\n")

cols0 <- copy(colnames(dfuture))
filas <- nrow(dfuture)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dfuture[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dfuture), cols0 ) )
  setcolorder( dfuture, c( cols_canaritos, cols0 ) )
}

In [ ]:
# dataset de future

mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

In [ ]:
# Inicializar predicciones
cat("Aplicando ensemble de", PARAM$train_final$APO * PARAM$train_final$ksemillerio, "modelos...\n")

vpred_acum <- rep(0.0, nrow(dfuture))
qacumulados <- 0

for( sem in PARAM$train_final$semillas ) {
  
  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( file.exists( arch_modelo ) )
  {
    cat("Aplicando modelo semilla", sem, "...\n")
    modelo_final <- lgb.load(arch_modelo)
    vpred_acum <- vpred_acum + predict(modelo_final, mfuture)
    qacumulados <- qacumulados + 1
    rm(modelo_final)
    gc()
  }
}

# Promedio de probabilidades
vpred_acum <- vpred_acum / qacumulados
cat("Ensemble completado. Modelos acumulados:", qacumulados, "\n")

# Crear tabla de predicción
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := vpred_acum]

# Ordenar por probabilidad descendente
setorder(tb_prediccion, -prob)

cat("Probabilidad máxima:", max(tb_prediccion$prob), "\n")
cat("Probabilidad mínima:", min(tb_prediccion$prob), "\n")
cat("Probabilidad mediana:", median(tb_prediccion$prob), "\n")

Sys.time()

In [ ]:
### Generación de Archivos de Producción

cat("\n=== GENERANDO ARCHIVOS DE PRODUCCIÓN ===\n")

dir.create("produccion", showWarnings=FALSE)

for( envios in PARAM$train_final$envios_produccion ) {
  
  # Seleccionar top N clientes
  clientes_a_contactar <- tb_prediccion[1:envios, numero_de_cliente]
  
  # Nombre del archivo (CSV)
  archivo_produccion <- paste0("./produccion/", PARAM$experimento, "_", envios, ".csv")
  
  # IMPORTANTE: Grabar SIN ENCABEZADO (col.names = FALSE)
  fwrite(
    data.table(numero_de_cliente = clientes_a_contactar),
    file = archivo_produccion,
    col.names = FALSE,  # SIN ENCABEZADO
    sep = ","           # SEPARADO POR COMAS (aunque es una sola columna)
  )
  
  cat("Archivo generado:", archivo_produccion, "(", envios, "clientes )\n")
}

cat("\n=== RESUMEN DE ARCHIVOS GENERADOS ===\n")
cat("Carpeta: ./produccion/\n")
cat("Cantidad de archivos:", length(PARAM$train_final$envios_produccion), "\n")
cat("Rango de envíos:", min(PARAM$train_final$envios_produccion), "-", 
    max(PARAM$train_final$envios_produccion), "\n")

# Guardar predicciones completas para análisis posterior
archivo_predicciones_completo <- paste0("./produccion/predicciones_completas_", 
                                        PARAM$experimento, ".csv")
fwrite(
  tb_prediccion,
  file = archivo_predicciones_completo,
  sep = ","
)

cat("Predicciones completas guardadas en:", archivo_predicciones_completo, "\n")

### Verificación de Archivos

cat("\n=== VERIFICACIÓN DE FORMATO ===\n")

# Verificar un archivo de ejemplo
archivo_ejemplo <- paste0("./produccion/", PARAM$experimento, "_", 
                          PARAM$train_final$envios_produccion[1], ".csv")

if(file.exists(archivo_ejemplo)) {
  cat("Verificando archivo:", archivo_ejemplo, "\n")
  
  # Leer primeras líneas
  primeras_lineas <- readLines(archivo_ejemplo, n = 5)
  cat("Primeras 5 líneas del archivo:\n")
  cat(paste(primeras_lineas, collapse = "\n"), "\n")
  
  # Verificar cantidad de líneas
  todas_lineas <- readLines(archivo_ejemplo)
  cat("\nTotal de líneas en archivo:", length(todas_lineas), "\n")
  
  # Verificar formato (solo números)
  tiene_solo_numeros <- all(grepl("^[0-9]+$", todas_lineas))
  cat("Formato correcto (solo números):", tiene_solo_numeros, "\n")
  
  if(!tiene_solo_numeros) {
    cat("ADVERTENCIA: El archivo contiene caracteres no numéricos\n")
  }
}

cat("\n=== ANÁLISIS DE DISTRIBUCIÓN DE PROBABILIDADES ===\n")

# Crear cuantiles
cuantiles <- quantile(tb_prediccion$prob, probs = seq(0, 1, 0.1))
cat("Deciles de probabilidad:\n")
print(cuantiles)

# Resumen estadístico
cat("\nResumen de probabilidades:\n")
print(summary(tb_prediccion$prob))

Sys.time()

cat("\n✓ PROCESO COMPLETADO\n")
cat("Archivos listos para campaña de retención en carpeta: ./produccion/\n")